In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [ ]:
!pip install gym
!pip install stable_baselines3[extra]

In [ ]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

# 3. Building an Environment

In [ ]:
class ScenarioEnv(Env):
    def __init__(self):
        # Actions we can take: 0 - Do Nothing, 1 - Launch
        self.action_space = Discrete(7)
        # Target Damage state array: 0 - Untouched, 1 - Disabled, 2 - Destroyed
        self.observation_space = MultiDiscrete([14, 14, 14, 14, 14, 14])
        # self.observation_space = Box(np.array([0]), np.array([1]),dtype= np.int)
        # Set start damage
        self.state = 0
        # Set number of missles
        self.missle_count = 14

    def step(self, action):
        done = False

        # Apply action
        if action == 0:
            reward = 0
        elif action == 1:
            reward = 1
            self.state[0] = self.state[0] + 1
            if self.state[0] > 2:
                reward = -5
        elif action == 2:
            reward = 1
            self.state[1] = self.state[1] + 1
            if self.state[1] > 1:
                reward = -5
        elif action == 3:
            reward = 1
            self.state[2] = self.state[2] + 1
            if self.state[2] > 1:
                reward = -5
        elif action == 4:
            reward = 1
            self.state[3] = self.state[3] + 1
            if self.state[3] > 1:
                reward = -5
        elif action == 5:
            reward = 0
            self.state[4] = self.state[4] + 1
        elif action == 6:
            reward = 0
            self.state[5] = self.state[5] + 1
        else:
            reward = 0

        if self.state[0] == 2 and self.state[1] == 1 and self.state[2] == 1 and self.state[3] == 1:
            reward += 100
            done = True

        # Reduce shower length by 1 second
        self.missle_count -= 1

        # Check if shower is done
        if self.missle_count <= 0:
            done = True

        # Apply temperature noise
        # self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        # Reset shower temperature
        self.state = np.array([0, 0, 0, 0, 0, 0], dtype=np.int64)
        # Reset shower time
        self.missle_count = 10
        return self.state


In [ ]:
env=ScenarioEnv()

In [ ]:
env.reset()

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(env, warn=True)

# 4. Test Environment

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [ ]:
env.close()

# 5. Train Model

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=50000)

# 6. Save Model

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=1000, render=False)

In [ ]:
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
        print('Episode:{} Score:{} Action:{} State:{}'.format(episode, score, action, obs))
env.close()